# Retrieval-Augmented Generation

## Setting up the LLM

In [ ]:
!pip install --upgrade langchain
!pip install --upgrade langchain-core
!pip install --upgrade langchain-community
!pip install --upgrade langchain-google-genai

#### First Test

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

# 1. Initialize the Chat Model (from the previous step)
chat_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                                  temperature=0,
                                  google_api_key=userdata.get('google_api_key'))

# 2. Prepare the messages
# The SystemMessage sets the behavior and context for the AI.
# The HumanMessage is the user's actual query.
messages = [
    SystemMessage(content="You're an assistant knowledgeable about healthcare. Only answer healthcare-related questions."),
    HumanMessage(content="What is Ayushman Bharat?"),
]

# 3. Invoke the model with the messages
result = chat_model.invoke(messages) # notice the similarity with model.predict from sklearn

print(result.content)

Ayushman Bharat is a flagship national health protection scheme launched by the Government of India in 2018. Its primary goal is to achieve Universal Health Coverage (UHC) and ensure that no one is left behind due to financial constraints when seeking healthcare.

The scheme has two main components:

1.  **Pradhan Mantri Jan Arogya Yojana (PMJAY):** This is the health insurance component, often referred to as the world's largest government-funded health insurance scheme.
    *   **Objective:** To provide financial protection to over 50 crore (500 million) poor and vulnerable families for secondary and tertiary care hospitalization.
    *   **Coverage:** It offers a health cover of up to ₹5 lakh (approximately $6,000 USD) per family per year for cashless and paperless treatment at empanelled public and private hospitals across India.
    *   **Benefits:** Covers pre-hospitalization expenses, hospitalization expenses, and post-hospitalization expenses for a wide range of medical and surg

In [ ]:
chat_model.invoke("What is blood pressure?")

AIMessage(content='Blood pressure is the **force of your blood pushing against the walls of your arteries** as your heart pumps it throughout your body.\n\nThink of your heart as a pump and your arteries as a network of hoses. When the pump pushes blood, it creates pressure inside the hoses. This pressure is essential to deliver oxygen and nutrients to all your organs and tissues.\n\n**How it\'s Measured (The Two Numbers):**\n\nBlood pressure is expressed as two numbers, typically written as a fraction (e.g., 120/80 mmHg):\n\n1.  **Systolic Pressure (the top number):** This is the higher number and represents the pressure in your arteries when your heart **beats** (contracts) and pushes blood out.\n2.  **Diastolic Pressure (the bottom number):** This is the lower number and represents the pressure in your arteries when your heart is **at rest** (between beats) and filling with blood.\n\nThe unit "mmHg" stands for millimeters of mercury, which is the standard unit for measuring pressure

#### Second Test

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

# 1. Initialize the Chat Model
chat_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                                  temperature=0,
                                  google_api_key=userdata.get('google_api_key'))

# 2. Prepare messages with an out-of-scope question
messages = [
    SystemMessage(content="You're an assistant knowledgeable about healthcare. Only answer healthcare-related questions."),
    HumanMessage(content="How do I change a tire?"),
]

# 3. Invoke the model
result = chat_model.invoke(messages)

print(result.content)

I cannot provide instructions on how to change a tire.  That's not within the scope of healthcare advice.  Changing a tire is a mechanical task and requires a separate set of instructions.  If you need to change a tire, I recommend consulting a trusted automotive resource or calling for roadside assistance.


## ChatPromptTemplate

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. Initialize the Chat Model
chat_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                                  temperature=0,
                                  google_api_key=userdata.get('google_api_key'))

# 2. Create the Prompt Template
instruction_str = """Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed as possible, but don't make up any information that's not from the context.
If you don't know an answer, say you don't know.

Context: {context}

Question: {question}
"""

review_template = ChatPromptTemplate.from_template(instruction_str)

# 3. Define the context and question
context = "The discharge process was seamless!"
question = "Did anyone have a positive experience?"

# 4. Create the chain by piping the components together
#    We also add an output parser to get a clean string result.
chain = review_template | chat_model | StrOutputParser()

# 5. Invoke the chain with the input variables
result = chain.invoke({
    "context": context,
    "question": question
})

print(result)

Yes, at least one patient had a positive experience.  One review specifically mentions that the discharge process was seamless.


## Using PromptTemplates & MessageTemplates

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)
from langchain_core.output_parsers import StrOutputParser

# 1. Initialize the Chat Model
chat_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                                  temperature=0,
                                  google_api_key=userdata.get('google_api_key'))

# 2. Create the detailed prompt templates
instruction_str = """Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions.
Be as detailed as possible, but don't make up any information that's not from the context.
If you don't know an answer, say you don't know.

Context: {context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=instruction_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]

# This is our final, reusable prompt template
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)

# 3. Define the context and question
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

# 4. Create the chain
chain = review_prompt_template | chat_model | StrOutputParser()

# 5. Invoke the chain
result = chain.invoke({
    "context": context,
    "question": question
})

print(result)

Yes, one patient stated, "I had a great stay!"


In [ ]:
context = "I had a negative stay!"
question = "Did anyone have a positive experience?"

chain.invoke({"context": context, "question": question})

'Based on the provided context, it only states, "I had a negative stay!" There is no information about anyone having a positive experience.'

# Adding RAG

In [ ]:
!pip install -U langchain-chroma # FAISS

In [6]:
import time
from google.colab import userdata

# Import the CSVLoader class to load documents from a CSV file.
from langchain.document_loaders.csv_loader import CSVLoader

# Import the Chroma class, which is used to create and interact with a Chroma vector database.
from langchain_chroma import Chroma

# Import the GoogleGenerativeAIEmbeddings class to create numerical vector representations (embeddings) of text using Google's models.
from langchain_google_genai import GoogleGenerativeAIEmbeddings


REVIEWS_CSV_PATH = "reviews.csv"

# Define a constant variable for the directory where the Chroma vector database will be stored.
REVIEWS_CHROMA_PATH = "chroma_data"


# Create an instance of the CSVLoader.
loader = CSVLoader(
    file_path=REVIEWS_CSV_PATH,  # Specify the path to the CSV file to be loaded.
    source_column="review"       # Specify the name of the column that contains the main text content.
)

# Call the .load() method on the loader instance.
# This reads the specified column from the CSV file and loads the content into a list of Document objects.
reviews = loader.load()

# Specify the embedding function to use. We define it once to be reused.
embedding_function = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",  # Choose the specific embedding model provided by Google.
    google_api_key=userdata.get('GOOGLE_API_KEY')  # Securely fetch the Google API key.
)

# Set the size of each batch to process.
batch_size = 20
# Calculate the total number of batches.
num_batches = (len(reviews) - 1) // batch_size + 1
reviews_vector_db = None

# Loop through the documents in batches to avoid hitting the API's rate limit.
for i in range(0, len(reviews), batch_size):
    # Get the current batch of documents.
    batch_docs = reviews[i:i + batch_size]
    current_batch_num = i // batch_size + 1

    print(f"Processing batch {current_batch_num}/{num_batches}...")

    if i == 0:
        # For the first batch, create a new Chroma vector database.
        # The `from_documents` method handles the entire process of embedding and storing the data.
        reviews_vector_db = Chroma.from_documents(
            documents=batch_docs,  # Pass the list of Document objects that need to be embedded.
            embedding=embedding_function,
            # Specify the directory on the disk where the vector database will be saved.
            # This makes the database persistent, so we can load it directly in the future.
            persist_directory=REVIEWS_CHROMA_PATH
        )
    else:
        # For subsequent batches, add the documents to the existing database.
        reviews_vector_db.add_documents(documents=batch_docs)

    # Pause the script for 30 seconds after each batch to respect the per-minute rate limit.
    print(f"Batch {current_batch_num} processed. Waiting for 30 seconds...")
    time.sleep(30)

print("Vector database created successfully and saved to the specified directory.")

Processing batch 1/51...
Batch 1 processed. Waiting for 30 seconds...
Processing batch 2/51...
Batch 2 processed. Waiting for 30 seconds...
Processing batch 3/51...
Batch 3 processed. Waiting for 30 seconds...
Processing batch 4/51...
Batch 4 processed. Waiting for 30 seconds...
Processing batch 5/51...
Batch 5 processed. Waiting for 30 seconds...
Processing batch 6/51...
Batch 6 processed. Waiting for 30 seconds...
Processing batch 7/51...
Batch 7 processed. Waiting for 30 seconds...
Processing batch 8/51...
Batch 8 processed. Waiting for 30 seconds...
Processing batch 9/51...
Batch 9 processed. Waiting for 30 seconds...


KeyboardInterrupt: 

## Retrieval

In [6]:
question = """Has anyone complained about communication with the hospital staff?"""
relevant_chunks = reviews_vector_db.similarity_search(question, k=3)

relevant_chunks[0].page_content

"review_id: 707\nvisit_id: 4533\nreview: I encountered some issues with the nursing staff's communication. It seemed like there was a lack of coordination, leading to confusion about my medication schedule and treatment plan.\nphysician_name: Joseph Gonzales\nhospital_name: Brown-Golden\npatient_name: Makayla Reynolds"

In [ ]:
relevant_chunks[1].page_content

"review_id: 707\nvisit_id: 4533\nreview: I encountered some issues with the nursing staff's communication. It seemed like there was a lack of coordination, leading to confusion about my medication schedule and treatment plan.\nphysician_name: Joseph Gonzales\nhospital_name: Brown-Golden\npatient_name: Makayla Reynolds"

In [ ]:
relevant_chunks[2].page_content

'review_id: 43\nvisit_id: 2353\nreview: The hospital staff lacked proper communication among themselves, leading to confusion about my treatment plan. Clear and cohesive communication is essential for patient care and overall satisfaction.\nphysician_name: Erika Ingram\nhospital_name: Shea LLC\npatient_name: Dennis Fitzgerald'

In [ ]:
from langchain.schema.runnable import RunnablePassthrough  # Allows passing inputs through unchanged in a pipeline
from langchain_core.output_parsers import StrOutputParser  # Parses the model's output into a clean string

# Create a retriever to fetch the top 10 most relevant reviews based on a query
reviews_retriever = reviews_vector_db.as_retriever(k=10)
# The `as_retriever` method converts the database into a retriever.
# `k=10` specifies that the retriever should return the top 10 most relevant documents for a query.

# Create a chain for querying and generating responses
review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    # Step 1: Retrieves relevant reviews (`context`) and passes the `question` unchanged
    | review_prompt_template
    # Step 2: Formats the retrieved reviews and the user's question into a structured prompt
    | chat_model
    # Step 3: Sends the prompt to the Gemini chat model to generate a response
    | StrOutputParser()
    # Step 4: Parses the model's raw output into a clean string format for easier use
)

In [ ]:
question = """Has anyone complained about communication with the hospital staff?"""
review_chain.invoke(question)

"Yes, patients have complained about communication with the hospital staff.\n\n*   **Makayla Reynolds** (review_id: 707) encountered issues with the nursing staff's communication at Brown-Golden hospital, noting a lack of coordination that led to confusion about her medication schedule and treatment plan.\n*   **Dennis Fitzgerald** (review_id: 43) stated that the hospital staff at Shea LLC lacked proper communication among themselves, which caused confusion about his treatment plan. He emphasized that clear and cohesive communication is essential for patient care and overall satisfaction."

# Putting it all together

In [1]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

In [2]:
from langchain.prompts import (
    PromptTemplate,  # Template for formatting prompts with dynamic variables
    SystemMessagePromptTemplate,  # Represents a system-level instruction to the model
    HumanMessagePromptTemplate,  # Represents a human-level input for the model
    ChatPromptTemplate,  # Combines multiple prompt components into a unified chat prompt
)

# Define the system prompt template as a string with placeholders for dynamic content
review_template_str = """Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions.
Be as detailed as possible, but don't make up any information that's not from the context.
If you don't know an answer, say you don't know.

{context}
"""

# Create a system-level message prompt template for the chatbot
review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"],  # Placeholder for the "context" (e.g., patient reviews)
        template=review_template_str,  # The instructions and structure of the system prompt
    )
)

# Create a human-level message prompt template for user input
review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],  # Placeholder for the "question" to be answered
        template="{question}",  # A simple template where the "question" is dynamically inserted
    )
)

# Combine the system and human prompts into a list of messages
messages = [review_system_prompt, review_human_prompt]

# Create a chat prompt template that integrates the system and human prompts
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],  # Define the expected inputs for the template
    messages=messages,  # Combine the individual prompt components (system and human)
)

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
chat_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                                  temperature=0,
                                  google_api_key=userdata.get('GOOGLE_API_KEY_2'))

In [17]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

# Specify the embedding function to use. We define it once to be reused.
embedding_function = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",  # Choose the specific embedding model provided by Google.
    google_api_key=userdata.get('GOOGLE_API_KEY_2')  # Securely fetch the Google API key.
)

reviews_vector_db = Chroma(
    persist_directory="chroma_data",
    embedding_function=embedding_function
)

In [ ]:
# Importing required modules and classes
from langchain.document_loaders.csv_loader import CSVLoader
import time

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import (
    PromptTemplate,  # Template for structuring prompts
    SystemMessagePromptTemplate,  # System-level instructions for the model
    HumanMessagePromptTemplate,  # Human input instructions for the model
    ChatPromptTemplate,  # Combines system and human prompts into a single chat prompt
)
from langchain_core.output_parsers import StrOutputParser  # Parses the model's output into a clean string
from langchain_community.vectorstores import Chroma  # Vector database for efficient similarity searches
from langchain_google_genai import GoogleGenerativeAIEmbeddings  # Converts text to embeddings using Google's API
from langchain.schema.runnable import RunnablePassthrough  # Allows passing inputs through unchanged in a pipeline

# Path to the persistent Chroma vector database
REVIEWS_CSV_PATH = "reviews.csv"

# Define a constant variable for the directory where the Chroma vector database will be stored.
REVIEWS_CHROMA_PATH = "chroma_data"


# Create an instance of the CSVLoader.
loader = CSVLoader(
    file_path=REVIEWS_CSV_PATH,  # Specify the path to the CSV file to be loaded.
    source_column="review"       # Specify the name of the column that contains the main text content.
)

# Call the .load() method on the loader instance.
# This reads the specified column from the CSV file and loads the content into a list of Document objects.
reviews = loader.load()

# Specify the embedding function to use. We define it once to be reused.
embedding_function = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-004",  # Choose the specific embedding model provided by Google.
    google_api_key=userdata.get('GOOGLE_API_KEY_2')  # Securely fetch the Google API key.
)

# Set the size of each batch to process.
batch_size = 20
# Calculate the total number of batches.
num_batches = (len(reviews) - 1) // batch_size + 1
reviews_vector_db = None

# Loop through the documents in batches to avoid hitting the API's rate limit.
for i in range(0, len(reviews), batch_size):
    # Get the current batch of documents.
    batch_docs = reviews[i:i + batch_size]
    current_batch_num = i // batch_size + 1

    print(f"Processing batch {current_batch_num}/{num_batches}...")

    if i == 0:
        # For the first batch, create a new Chroma vector database.
        # The `from_documents` method handles the entire process of embedding and storing the data.
        reviews_vector_db = Chroma.from_documents(
            documents=batch_docs,  # Pass the list of Document objects that need to be embedded.
            embedding=embedding_function,
            # Specify the directory on the disk where the vector database will be saved.
            # This makes the database persistent, so we can load it directly in the future.
            persist_directory=REVIEWS_CHROMA_PATH
        )
    else:
        # For subsequent batches, add the documents to the existing database.
        reviews_vector_db.add_documents(documents=batch_docs)

    # Pause the script for 60 seconds after each batch to respect the per-minute rate limit.
    print(f"Batch {current_batch_num} processed. Waiting for 30 seconds...")
    time.sleep(30)

print("Vector database created successfully and saved to the specified directory.")

In [18]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Create a retriever to fetch the top 10 most relevant reviews based on a query
reviews_retriever = reviews_vector_db.as_retriever(k=10)
# The `as_retriever` method converts the database into a retriever.
# `k=10` specifies that the retriever should return the top 10 most relevant documents for a query.

# Create a chain for querying and generating responses
review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    # Step 1: Retrieves relevant reviews (`context`) and passes the `question` unchanged
    | review_prompt_template
    # Step 2: Formats the retrieved reviews and the user's question into a structured prompt
    | chat_model
    # Step 3: Sends the prompt to the OpenAI chat model to generate a response
    | StrOutputParser()
    # Step 4: Parses the model's raw output into a clean string format for easier use
)

In [9]:
# question = """Has anyone complained about communication with the hospital staff?"""
# review_chain.invoke(question)

# Adding a UI

In [ ]:
!pip install gradio

In [15]:
def respond_to_user_question(question: str, history: list) -> str:
    """
    Respond to a user's question using the review_chain.
    """
    return review_chain.invoke(question)

In [12]:
# respond_to_user_question("Has anyone complained about communication with the hospital staff?", [])

In [19]:
import gradio as gr
# Create the Gradio ChatInterface
interface = gr.ChatInterface(fn=respond_to_user_question, title="Review Helper Bot")

# Launch the Gradio app
interface.launch(debug=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6685db31ab6ec72011.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6685db31ab6ec72011.gradio.live
